# 1. Install Necessary Libraries

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Initialize MediaPipe for Keypoint Detection

In [ ]:
mp_model = mp.solutions.holistic # Holistic model
mp_utils = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
# Define a function to process the image and predict keypoints
def detect_keypoints(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert color from BGR to RGB
    image.flags.writeable = False                  # Set image to non-writeable
    results = model.process(image)                 # Perform prediction
    image.flags.writeable = True                   # Set image back to writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Convert color back to BGR
    return image, results

In [ ]:
# Define a function to draw keypoints on the image
def draw_keypoints(image, results):
    mp_utils.draw_landmarks(image, results.face_landmarks, mp_model.FACE_CONNECTIONS) # Face keypoints
    mp_utils.draw_landmarks(image, results.pose_landmarks, mp_model.POSE_CONNECTIONS) # Pose keypoints
    mp_utils.draw_landmarks(image, results.left_hand_landmarks, mp_model.HAND_CONNECTIONS) # Left hand keypoints
    mp_utils.draw_landmarks(image, results.right_hand_landmarks, mp_model.HAND_CONNECTIONS) # Right hand keypoints

In [ ]:
# Define a function to draw styled landmarks with customized colors and thickness
def draw_styled_keypoints(image, results):
    # Drawing settings for each type of keypoints with specific colors and thickness
    mp_utils.draw_landmarks(image, results.face_landmarks, mp_model.FACE_CONNECTIONS, 
                             mp_utils.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_utils.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    mp_utils.draw_landmarks(image, results.pose_landmarks, mp_model.POSE_CONNECTIONS,
                             mp_utils.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_utils.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
    mp_utils.draw_landmarks(image, results.left_hand_landmarks, mp_model.HAND_CONNECTIONS, 
                             mp_utils.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_utils.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
    mp_utils.draw_landmarks(image, results.right_hand_landmarks, mp_model.HAND_CONNECTIONS, 
                             mp_utils.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_utils.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))


In [ ]:
# Setup webcam for real-time video capture
camera = cv2.VideoCapture(0)

In [ ]:
# Use MediaPipe Holistic model with confidence thresholds
with mp_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while camera.isOpened():
        ret, frame = camera.read()  # Read frame from the webcam
        image, results = detect_keypoints(frame, holistic)  # Detect keypoints
        print(results)  # Optionally print the detection results

        draw_styled_keypoints(image, results)  # Draw styled keypoints

        cv2.imshow('Live Feed', image)  # Display the processed image

        if cv2.waitKey(10) & 0xFF == ord('q'):  # Exit loop if 'q' is pressed
            break

    camera.release()  # Release the camera
    cv2.destroyAllWindows()  # Close all OpenCV windows
draw_keypoints(frame, results)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values for Use in ML Models

In [ ]:
pose_keypoints = []
if results.pose_landmarks:
    pose_keypoints = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten()

face_keypoints = np.array([[lm.x, lm.y, lm.z] for lm in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
left_hand_keypoints = np.array([[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
right_hand_keypoints = np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)

In [ ]:
# Function to extract all keypoints into a single vector
def extract_keypoints(results):
    pose = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[lm.x, lm.y, lm.z] for lm in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([pose, face, lh, rh])

full_keypoints = extract_keypoints(results)  # Extract and concatenate all keypoints
np.save('keypoints_array', full_keypoints)  # Save keypoints to a file
loaded_keypoints = np.load('keypoints_array.npy')  # Load keypoints from the file

# 4. Setup Data Collection Folders

In [ ]:
data_directory = os.path.join('Keypoints_Data')  # Path for exported data
actions = np.array(['hello', 'thanks', 'iloveyou'])  # Actions to detect
num_sequences = 30  # Number of video sequences per action
sequence_length = 30  # Length of each video sequence
initial_folder = 30  # Starting folder number

In [ ]:
# Create folders for each action and sequence
for action in actions:
    action_path = os.path.join(data_directory, action)
    if not os.path.exists(action_path):
        os.makedirs(action_path)
    for sequence in range(1, num_sequences + 1):
        sequence_path = os.path.join(action_path, str(initial_folder + sequence))
        if not os.path.exists(sequence_path):
            os.makedirs(sequence_path)

# 5. Collect Keypoint Data for Each Action Sequence

In [ ]:
camera = cv2.VideoCapture(0)  # Initialize the webcam
with mp_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence_num in range(initial_folder, initial_folder + num_sequences):
            for frame_count in range(sequence_length):
                ret, frame = camera.read()
                image, results = detect_keypoints(frame, holistic)  # Detect keypoints
                draw_styled_keypoints(image, results)  # Draw keypoints with styles

                if frame_count == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, f'Collecting frames for {action} Video Number {sequence_num}', (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('Live Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, f'Collecting frames for {action} Video Number {sequence_num}', (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('Live Feed', image)

                keypoints = extract_keypoints(results)  # Extract keypoints from the current frame
                npy_path = os.path.join(data_directory, action, str(sequence_num), str(frame_count))
                np.save(npy_path, keypoints)  # Save keypoints data

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

camera.release()
cv2.destroyAllWindows()

# 6. Prepare Data for Model Training

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
# Create a mapping from action labels to integers
action_map = {action: idx for idx, action in enumerate(actions)}

In [ ]:
# Collect all sequences and their corresponding labels
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(data_directory, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(data_directory, action, str(sequence), f"{frame_num}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(action_map[action])

X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# 7. Construct and Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
# Define a sequential model with LSTM layers and dense layers
model = Sequential([
    LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)),
    LSTM(128, return_sequences=True, activation='relu'),
    LSTM(64, return_sequences=False, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

In [ ]:
# Compile the model with Adam optimizer and categorical crossentropy loss
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
# Training the model with TensorBoard callbacks for visualization
tb_callback = TensorBoard(log_dir=os.path.join('Logs'))
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])
model.summary()

# 8. Evaluate Model with Test Data

In [ ]:
predicted = model.predict(X_test)  # Predict the test data
predicted_actions = [actions[np.argmax(res)] for res in predicted]
true_actions = [actions[np.argmax(res)] for res in y_test]

# 9. Save and Load the Model Weights

In [ ]:
model.save('action_model.h5')
del model  # Delete the current model instance
model.load_weights('action_model.h5')  # Load the saved model weights

# 10. Model Evaluation Using Confusion Matrix and Accuracy Metrics

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
predicted_labels = np.argmax(model.predict(X_test), axis=1)
true_labels = np.argmax(y_test, axis=1)
confusion = multilabel_confusion_matrix(true_labels, predicted_labels)
accuracy = accuracy_score(true_labels, predicted_labels)

# 11. Real-Time Action Recognition

In [ ]:
from scipy import stats

def visualize_probabilities(results, actions, frame, colors):
    """This function visualizes the prediction probabilities on the video feed."""
    output_frame = frame.copy()
    for num, prob in enumerate(results):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return output_frame

sequence = []  # To hold the last 30 frames
predictions = []  # To hold the last predictions
sentence = []  # To hold the recognized actions
threshold = 0.5  # Probability threshold for action recognition

camera = cv2.VideoCapture(0)
with mp_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while camera.isOpened():
        ret, frame = camera.read()
        image, results = detect_keypoints(frame, holistic)
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]  # Keep the last 30 frames

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            if np.unique(predictions[-10:])[0] == np.argmax(res) and res[np.argmax(res)] > threshold:
                if not sentence or actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            sentence = sentence[-5:]  # Keep last 5 recognized actions

            image = visualize_probabilities(res, actions, image, colors)
        cv2.imshow('Live Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

camera.release()
cv2.destroyAllWindows()